# Week 9

## Notes

### Random library

In [1]:
import random

In [9]:
random.random()

0.9027595269309289

In [28]:
def coin_toss(p = 0.5):
    if random.random() < p:
        return 'H'
    else:
        return 'T'

In [29]:
for _ in range(10):
    print(coin_toss(0.9))

H
T
H
T
H
H
H
H
H
H


In [34]:
random.randint(0,10)

10

In [35]:
l = [0.5,4,27,'apple',53.8]

In [43]:
random.choice(l)

'apple'

In [44]:
random.seed(20)

In [45]:
random.random()

0.9056396761745207

### `np.roll`

In [1]:
import numpy as np

In [2]:
A = np.reshape(np.arange(0,25),(5,5))
print(A)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


In [3]:
np.roll(A,(-1,0),(0,1))

array([[ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [ 0,  1,  2,  3,  4]])

In [8]:
A = np.zeros((6,6))
A[1:5,1:5] = 1
print(A)

[[0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [62]:
B = np.zeros((6,6))
for i in range(-1,2):
    for j in range(-1,2):
        B += np.roll(A,(i,j),(0,1))
B -= A
B[1:5,1:5]

array([[3., 5., 5., 3.],
       [5., 8., 8., 5.],
       [5., 8., 8., 5.],
       [3., 5., 5., 3.]])

**Exercise:** Write a function that takes an array as input and returns an array where each entry is the neighbor sum from the input array, calculated using `np.roll`.

In [58]:
def neighbor_sum(arr):
    i,j = np.shape(arr)
    border_arr = np.zeros((i+2,j+2),dtype=int)
    border_arr[1:i+1,1:j+1] = arr
    sum_arr = np.zeros((i+2,j+2),dtype=int)
    for r in range(-1,2):
        for c in range(-1,2):
            sum_arr += np.roll(border_arr,(r,c),(0,1))
    return sum_arr[1:i+1,1:j+1] - arr

In [17]:
A = np.reshape(np.arange(0,9),(3,3))
print(A)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [20]:
np.roll(A,(-1,1),(0,1))

array([[5, 3, 4],
       [8, 6, 7],
       [2, 0, 1]])

In [19]:
np.roll(A,(-1,0),(0,1))

array([[3, 4, 5],
       [6, 7, 8],
       [0, 1, 2]])

In [21]:
np.roll(A,(-1,-1),(0,1))

array([[4, 5, 3],
       [7, 8, 6],
       [1, 2, 0]])

### Counting neighbors of a particular value (arithmetically)

In [23]:
B = neighbor_sum(np.ones((5,5)))
print(B)

[[3. 5. 5. 5. 3.]
 [5. 8. 8. 8. 5.]
 [5. 8. 8. 8. 5.]
 [5. 8. 8. 8. 5.]
 [3. 5. 5. 5. 3.]]


In [30]:
fives = 1-np.ceil(np.absolute((B-5)/10))

In [31]:
neighbor_sum(fives)

array([[2., 2., 2., 2., 2.],
       [2., 4., 3., 4., 2.],
       [2., 3., 0., 3., 2.],
       [2., 4., 3., 4., 2.],
       [2., 2., 2., 2., 2.]])

### Counting neighbors of a particular value (logically)

In [46]:
bool_arr = B == 5
bool_arr

array([[False,  True,  True,  True, False],
       [ True, False, False, False,  True],
       [ True, False, False, False,  True],
       [ True, False, False, False,  True],
       [False,  True,  True,  True, False]])

In [47]:
np.array(bool_arr,dtype = int)

array([[0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0]])

### Conway's Game of Life

Array of 0s and 1s with the 1s representing living cells and the 0s non-living cells.

Rules for how life evolves
* Living cells with fewer than 2 live neighbors will die
* Living cells with 4 or more live neighbors will die
* non-living cells with exactly 3 live neighbors will come to life

In [71]:
def Conway(arr):
    living_neighbors = neighbor_sum(arr)
    arr = (arr + np.array((np.array(living_neighbors < 2,dtype=int) + A) == 2,dtype=int) 
           + np.array((np.array(living_neighbors > 3,dtype=int) + A) == 2,dtype=int)
           + np.array((np.array(living_neighbors == 3,dtype=int) - A) == 1,dtype=int))%2
    return arr

In [75]:
A = np.random.randint(0,2,(20,20))
A

array([[1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 1, 0,

In [81]:
A = Conway(A)
A

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0,